In [2]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.firefox.options import Options
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'hln_be_paid'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_download_image = 0
publish_source = 'hln.be'
country = 'Belgium'
language = 'Dutch'
images_path = []
foot_fall = c.get_foot_fall(publish_source)

rss = c.download_page('https://www.hln.be/net-binnen')

proxy_list = [ 'http://110.168.255.79',
              'http://103.31.251.17',
              'http://52.152.169.254',
              'http://5.166.57.222',
              'http://103.250.157.34',
              'http://190.152.71.230',
              'http://103.112.163.200',
              'http://187.216.90.46',
              'http://193.56.255.133',
              'http://91.107.6.115',
               ]

url = "https://www.hln.be/net-binnen"
r = requests.get(url)
authId = 'authId=' + r.cookies.get_dict()['authId']
headers = {
'Host': 'www.hln.be',
# 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'Accept-Language': 'en-US,en;q=0.5',
'Accept-Encoding': 'gzip, deflate, br',
'Connection': 'keep-alive',
'Cookie': authId,
'Upgrade-Insecure-Requests': "1",
'Cache-Control': 'max-age=0'
}
rss = requests.get(url, headers=headers).text
    
for i in rss.split('ankeiler--list-item">')[1:]:
    
    # source_link
    source_link = c.scrap('href="(.*?)"', i)
   
    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue
        
    time.sleep(random.randint(4,10))
    
    for i in range(3):
        logger.info(f'Fetching {source_link}\n')
    #         print(f'Fetching {source_link}\n')

        proxies = {
        'http': random.choice(proxy_list)
        }

        r = requests.get(url, proxies=proxies)
        headers['authId']='authId=' + r.cookies.get_dict()['authId']
        page = requests.get(source_link, headers=headers, proxies=proxies).text

        if 'Access Denied' not in page[:45]:
            break
        time.sleep(random.randint(100,300)) #300,500 
    
#     if page.startswith('Unable to fetch'):
#             logger.info(page) # writes error message with error code
#             unable_to_fetch_url += 1
#             continue  
    # Date and time
    pub_date, publish_time = '', ''

    try:
        date_time_str = c.scrap('datePublished":"(.*?)\+', page)      
        date_time_str = date_time_str.replace('T', ' ')
        date_time_str = re.sub('[^\w+]', '', date_time_str)  
        date_time_obj = datetime.strptime(date_time_str, '%Y%m%d%H%M%S') 
        ist_date_time = date_time_obj + timedelta(hours = 4,minutes = 30)  # utc+1 time to ist
        ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]
        
    except:
        pass

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')
#         print(f'Skipping due to date {source_link}\n')
        skipped_due_to_date += 1
        continue

    # break if date is not today's date
    if pub_date != created_on:
        break      
    
    # source_headline
    source_headline = c.scrap("<title>(.*?)\|", page) #&amp;
    source_headline = re.sub('&amp;','&',source_headline)
    source_headline = re.sub('&.*?;','',source_headline)
    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
#         print(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue
        
    # source_content         
    source_content = c.scrap('class="article__intro"\s*>(.*?)<div\s*class="article__meta">', page)
    if not source_content: 
        source_content = c.scrap('class="article__intro"\s*>(.*?)<div\s*class="article-login-gate', page)
    if not source_content: 
        source_content = c.scrap('class="article__paragraph">(.*?)<div\s*class="article__component article__component--snippet">', page)
    if not source_content: 
        source_content = c.scrap('class="article__paragraph">(.*?)<header\s*class="comments__header">', page)
    if not source_content:
        source_content = c.scrap('class="article__paragraph">(.*?)<div\s*class="article__component article__component--related">', page)
    source_content = re.sub('&amp;','&',source_content)
    source_content = re.sub('&.*?;','',source_content)
    source_content = c.strip_html(source_content)
  


    # skip if content not found
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')
        skipped_due_to_content += 1
        continue

    # journalist
    journalist = ''
    if not journalist: journalist = 'NA'

    # current date and time 
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []


    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":foot_fall,
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content
                } 

    cl_data.insert_one(clientdata)  # get object id and insert data
    no_of_data += 1


logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch url: {unable_to_fetch_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

https://www.hln.be/binnenland/niet-alleen-pfos-maar-ook-oorlogsmunitie-in-zwijndrechtse-bodem-tijd-voor-een-sanering~adab9d34/
https://www.hln.be/sport/live-1-1-shaqiri-straft-spaans-misverstand-af~adfa7aca/
https://www.hln.be/buitenland/toeriste-dood-teruggevonden-na-bezoek-aan-paradijselijk-strand-op-kreta~a797ab22/
https://www.hln.be/rode-duivels/live-weldra-kennen-we-het-antwoord-op-de-vraag-spelen-de-bruyne-en-hazard-duivels-in-het-rood~a1775cb3/
https://www.hln.be/tv/nagelbijten-tot-de-allerlaatste-vraag-dit-is-de-winnaar-van-blokken~a5b768b1/
https://www.hln.be/home/vlaming-kristof-van-de-voorde-heeft-vakantiehuizen-in-portugal-maar-kijkt-met-bang-hart-naar-oprukkende-deltavariant~a4627417/
https://www.hln.be/olympische-spelen/badmuts-voor-afrokapsel-niet-welkom-op-olympische-spelen~ad4506a6/
https://www.hln.be/voetbal/football-talk-toni-kroos-stopt-als-international-doelman-koffi-op-weg-naar-charleroi-avenatti-trekt-naar-promovendus-union~a17c7f4e/
https://www.hln.be/binnenland

https://www.hln.be/buitenland/el-chapo-van-azie-mag-worden-uitgeleverd-aan-australie-beslist-nederlandse-rechter~a092c25b/
https://www.hln.be/binnenland/oproep-kijk-jij-in-italie-naar-de-wedstrijd-vanavond~a083312d9/
https://www.hln.be/wimbledon/elise-mertens-neemt-het-in-de-derde-ronde-op-tegen-madison-keys-ja-ze-hebben-al-gevraagd-of-dries-mijn-broer-is~af4a6ff9/
https://www.hln.be/buitenland/australie-laat-vanaf-midden-juli-nog-maar-3-000-reizigers-per-week-binnen~a93fa5c1/
https://www.hln.be/auto/porsche-roept-43-000-elektrische-wagens-terug-wegens-softwareprobleem~a6216177/
https://www.hln.be/tour-de-france/tour-organisatie-laat-klacht-tegen-vrouw-met-het-bordje-vallen-rennersvakbond-zet-wel-door-francaise-moet-op-14-oktober-voor-rechter-verschijnen~ab199e12/
https://www.hln.be/buitenland/vierde-dag-op-rij-recordaantal-coronadoden-in-rusland~a9225b98/
https://www.hln.be/ek-voetbal/toch-het-grote-gelijk-van-enrique-spanje-vindt-juiste-flow-en-start-als-favoriet-tegen-stuntploeg-zwi

https://www.hln.be/showbizz/de-dood-van-zijn-vrouw-het-missen-van-publiek-en-belgie-italie-hoe-zou-het-nog-zijn-met-rocco-granata~a1611f43/
https://www.hln.be/binnenland/belgie-telde-4-000-minder-kankerdiagnoses-in-2020-maar-dat-is-geen-goed-nieuws~ab9495e1/
https://www.hln.be/showbizz/mondmaskers-bubbels-verdeeld-over-2-500-man-en-ontsmetting-a-volonte-festivalzomer-light-van-start~a4328ae2/
https://www.hln.be/dieren/duitse-dierentuin-is-een-rode-panda-kwijt~a2305eda/
https://www.hln.be/auto/duitsland-haalt-doel-van-1-miljoen-elektrische-auto-s-met-half-jaar-vertraging~a3cd362c/
https://www.hln.be/rode-duivels/italie-heeft-ferrari-versace-gucci-prada-vespa-wij-hebben-lukaku-onze-chef-voetbal-legt-alle-hoop-bij-spits-van-inter~a15eeeed/
https://www.hln.be/wetenschap-en-planeet/zestig-jaar-gewacht-en-nu-mag-ze-eindelijk-de-ruimte-in-naast-bezos-zit-straks-82-jarige-wally-funk~a4f1224f/
https://www.hln.be/reizen/strengere-maatregelen-voor-wie-terugkeert-uit-portugal-maar-welke~aaa023f2/
